Task 3: Джойны

In [8]:
import pandas as pd

In [9]:
from sqlite3 import connect

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [11]:
con = connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

Создайте новую таблицу datamart в базе данных, объединив таблицы pageviews и checker с помощью только одного запроса.
Таблица должна содержать следующие столбцы: uid, labname, first_commit_ts, first_view_ts.
first_commit_ts — это просто новое имя для столбца timestamp из таблицы checker; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
first_view_ts — первое посещение пользователем из таблицы pageviews, метка времени посещения пользователем ленты новостей.
По-прежнему нужно использовать фильтр status = 'ready'.
По-прежнему нужно использовать фильтр numTrials = 1.
Имена лабораторных заданий по-прежнему должны быть из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1.
Таблица должна содержать только пользователей (uid с user_*), а не администраторов.
first_commit_ts и first_view_ts должны быть распарсены как datetime64[ns].


In [16]:
datamart=pd.io.sql.read_sql('''SELECT c.uid,
                              c.labname,
                              min(c.timestamp) as first_commit_ts, 
                              min(datetime)  as first_view_ts
                          from checker c  left join pageviews p on p.uid=c.uid
                          where c.status = 'ready'  and c.numTrials = 1  and c.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
                            and c.uid like'%user%'
                            group by c.uid,
                              c.labname
''',con,parse_dates=['first_commit_ts', 'first_view_ts'])                
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


In [17]:
datamart.to_sql('datamart', 
                con, 
                if_exists='replace', 
                index=False
               )

Используя методы библиотеки Pandas, создайте два датафрейма: test и control.
test должен включать пользователей, у которых имеются значения в first_view_ts.
control должен включать пользователей, у которых отсутствуют значения в first_view_ts.
Замените пропущенные значения в control средним значением first_view_ts пользователей из test (оно пригодится нам для анализа в будущем).
Сохраните обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [18]:
test = datamart[datamart.first_view_ts.notna()];test.shape

(59, 4)

In [19]:
control = datamart[datamart.first_view_ts.isna()]; control.shape

(81, 4)

In [20]:
test_first_view_ts_mean = test.first_view_ts.mean()

In [21]:
control = control.fillna(value={'first_view_ts' :test_first_view_ts_mean},
                        axis=0)
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783552
137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783552
138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783552


In [22]:
test.to_sql('test', 
                con, 
                if_exists='replace', 
                index=False
               )

In [23]:
control.to_sql('control', 
                con, 
                if_exists='replace', 
                index=False
               )

Закройте соединение.

Проверка

In [25]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [27]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [31]:
con = connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

In [32]:
test=pd.io.sql.read_sql('select * from test',con)

In [33]:
control=pd.io.sql.read_sql('select * from control',con)